# Accessing  astronomical catalogs

There are two ways to access astronomical data catalogs that are provided as table data with a VO API.

First, there is a __[Simple Cone Search (SCS) protocol](http://www.ivoa.net/documents/latest/ConeSearch.html)__  used to search a given table with a given position and radius, getting back a table of results.  The interface requires only a position and search radius.   

For more complicated searches, the __[Table Access Protocol](http://www.ivoa.net/documents/TAP/)__ (TAP) protocol is  a powerful tool to search any VO table.  Here, we expand on its usage and that of the __[Astronomical Data Query Language](http://www.ivoa.net/documents/latest/ADQL.html)__ (ADQL) that it uses.  


* [1. Simple cone search](#cone) queries
* [2. Basic Table Access Protocol](#basic) queries
* [3 Expressing queries in ADQL](#adql)
* [4. Cross-correlating](#cc) our own catalog with a HEASARC catalog
* [5. Combining](#combo) data from multiple catalogs and cross-correlating



In [1]:
# suppress some specific warnings that are not important
import warnings
warnings.filterwarnings("ignore", module="astropy.io.votable.*")
warnings.filterwarnings("ignore", module="pyvo.utils.xml.*")

import io
import numpy as np

# Astropy imports
import astropy.units as u
import astropy.constants as const
from astropy.coordinates import SkyCoord
from astropy.cosmology import Planck15
from astropy.io import votable as apvot

## Generic VO access routines
import pyvo as vo


<a id="scs1"></a>

# 1. Simple cone search


Starting with a single simple source first: 

In [2]:
coord = SkyCoord.from_name("m51")
print(coord)

<SkyCoord (ICRS): (ra, dec) in deg
    (202.469575, 47.19525833)>


Below, we go through the exercise of how we can figure out the most relevant table. But for now, let's assume that we know that we want the CFA redshift catalog refered to as 'zcat'. VO services are listed in a central Registry that can be searched through a [web interface](http://vao.stsci.edu/keyword-search/) or using PyVO's `regsearch`.  We use the registry to find the corresponding cone service and then submit our cone search. 

Registry services are of the following type:  
* simple cone search:  "scs"
* table access protocol:  "tap" or "table"
* simple image search:  "sia" or "image"
* simple spectral access: "ssa"

There are a number of things in the registry related to 'zcat' so we find the specific one that we want, which is the CFA version:

In [3]:
services = vo.regsearch(servicetype='scs', keywords=['zcat'])
services.to_table()['ivoid', 'short_name', 'res_title']

ivoid,short_name,res_title
object,object,object
ivo://cds.vizier/j/apj/872/134,J/ApJ/872/134,"A uniformly selected, all-sky, optical AGN catalog (Zaw+, 2019)"
ivo://cds.vizier/j/apj/872/134,J/ApJ/872/134,"A uniformly selected, all-sky, optical AGN catalog (Zaw+, 2019)"
ivo://cds.vizier/j/apj/872/134,J/ApJ/872/134,"A uniformly selected, all-sky, optical AGN catalog (Zaw+, 2019)"
ivo://cds.vizier/j/mnras/339/652,J/MNRAS/339/652,The FLASH Redshift Survey (Kaldare+ 2003)
ivo://cds.vizier/vii/259,VII/259,"6dF galaxy survey final redshift release (Jones+, 2009)"
ivo://cds.vizier/vii/259,VII/259,"6dF galaxy survey final redshift release (Jones+, 2009)"
ivo://nasa.heasarc/sixdfgs,SIXDFGS,6dFGS Galaxy Survey Final Redshift Release Catalog
ivo://nasa.heasarc/uzc,UZC,Updated Zwicky Catalog
ivo://nasa.heasarc/zcat,CFAZ,CfA Redshift Catalog (June 1995 Version)


Supposing that we want the table with the short_name CFAZ, and we want to retrieve the data for all sources within an arcminute of our specified location:

(Note that currently, it's a bit ugly to retrieve the specific service we want.)  

In [4]:
## Use the one that's CFAZ. 
##  Use list comprehension to check each service's short_name attribute and use the first.
cfaz_cone_service = [s for s in services if 'CFAZ' in s.short_name][0] 

## We are searching for sources within 10 arcminutes of M51. 
results = cfaz_cone_service.search(pos=coord, radius=10*u.arcmin)
results.to_table()


name,ra,dec,bmag,radial_velocity,radial_velocity_error,redshift,class,Search_Offset
,deg,deg,,km / s,km / s,,,
object,float64,float64,float32,int32,int16,float64,int16,float64
N5194,202.46823,47.19815,9.03,474,23,--,6200,0.1819
N5195,202.49491,47.26792,10.94,558,23,--,6200,4.4802


The SCS is quite straightforward and returns all of the columns of the given table (which can be anything) for the sources in the region queried.

<a id="basic"></a>

# 2. Basic Table Access Protocol queries

A TAP query is the most powerful way to search a catalog. A Simple Cone Search only allows you to ask for a position and radius, but TAP allows you to do much more, since the available tables contain much more information. 



Many services list a single TAP service in the Registry that can access many catalogs, boosting your efficiency. This is the power of the TAP! 
With the TAP, you can refine the search based on any other attribute in the given catalog. Suppose for our example, we want to select bright galaxy candidates but don't know the coordinates. Therefore, we start from figuring out the best table to query.   

As before, we use the vo.regsearch() for a servicetype 'tap'.  There are a lot of TAP services in the registry, so we're going to limit our search to those related to the HEASARC.  

In [5]:
heasarc_tap_services = vo.regsearch(servicetype='tap', keywords=['heasarc'])
heasarc_tap_services.to_table()['ivoid', 'short_name', 'res_title']

ivoid,short_name,res_title
object,object,object
ivo://nasa.heasarc/services/xamin,HEASARC,HEASARC Xamin Catalog Interface


So there's only one HEASARC TAP service.  But it serves many different tables.  What are they?  This sends a query to the service to ask.  Then let's look for results related to the string 'zcat':

In [6]:
heasarc_tables = heasarc_tap_services[0].service.tables
for tablename in heasarc_tables.keys():
    if "zcat" in tablename:  
        heasarc_tables[tablename].describe()
        print("Columns={}".format(sorted([k.name for k in heasarc_tables[tablename].columns ])))
        print("----")

abellzcat
    Abell Clusters Measured Redshifts Catalog

Columns=['"__row"', '"__x_ra_dec"', '"__y_ra_dec"', '"__z_ra_dec"', 'abell_distance_class', 'abell_radius', 'abell_richness_class', 'bautz_morgan_class', 'bautz_morgan_flag', 'bii', 'class', 'dec', 'lii', 'log_redshift_ratio', 'mag_10', 'name', 'ra', 'redshift', 'redshift_flag', 'ref_redshift']
----
romabzcat
    Roma-BZCAT Multi-Frequency Catalog of Blazars

Columns=['"__row"', '"__x_ra_dec"', '"__y_ra_dec"', '"__z_ra_dec"', 'bii', 'dec', 'flux_143_ghz', 'flux_1p4_ghz', 'gr_photon_flux', 'lii', 'name', 'object_type', 'ra', 'redshift', 'redshift_flag', 'rmag', 'ro_spectral_index', 'source_number', 'xray_flux']
----
zcat
    CfA Redshift Catalog (June 1995 Version)

Columns=['"__row"', '"__x_ra_dec"', '"__y_ra_dec"', '"__z_ra_dec"', '"distance"', 'bar_type', 'bii', 'bmag', 'bt_mag', 'class', 'comments', 'dec', 'diameter_1', 'diameter_2', 'lii', 'luminosity_class', 'morph_type', 'name', 'notes', 'ra', 'radial_velocity', 'radial_vel

Success! This appears to be useful table for our goal since it contains columns with the information that we need to select a sample of the brightest nearby spiral galaxy candidates. 

Now that we know all the possible column information in the zcat catalog, we can do more than query on position (as in a cone search) but also on any other column (e.g., redshift, bmag, morph_type).  The query has to be expressed in a language called __[ADQL](http://www.ivoa.net/documents/latest/ADQL.html)__.  


<a id="adql"></a>

# 3. Expressing queries in ADQL


The basics of ADQL:

* *SELECT &#42; FROM my.interesting.catalog as cat...* 

says you want all ("&#42;") columns from the catalog called "my.interesting.catalog", which you will refer to in the rest of the query by the more compact name of "cat".  

Instead of returning all columns, you can 

* *SELECT cat.RA, cat.DEC, cat.bmag from catalog as cat...* 

to only return the columns you're interested in. To use multiple catalogs, your query could start, e.g.,

* *SELECT c1.RA,c1.DEC,c2.BMAG FROM catalog1 as c1 natural join catalog2 as c2...* 

says that you want to query two catalogs zipped together the "natural" way, i.e., by looking for a common column.

To select only some rows of the catalog based on the value in a column, you can add:  

* *WHERE cat.bmag < 14* 

says that you want to retrieve only those entries in the catalog whose bmag column has a value less than 14.

You can also append 

* *ORDER by cat.bmag* 

to return the result sorted ascending by one of the columns, adding *DESC* to the end for descending. 

A few special functions in the ADQL allow you to query regions:

* *WHERE contains( point('ICRS', cat.ra, cat.dec), circle('ICRS', 210.5, -6.5, 0.5))=1*

is how you would ask for any catalog entries whose RA,DEC lie within a circular region defined by RA,DEC 210.5,-6.5 and a radius of 0.5 (all in degrees).  The 'ICRS' specifies the coordinate system.  

See the ADQL documentation for more.

Here is a simple ADQL query where we print out the relevant columns for the bright (Bmag <14) sources found within 1 degree of M51 (we will discuss how to define the table and column names below):

In [7]:
##  Inside the format call, the {} are replaced by the given variables in order.
##  So this asks for 
##  rows of public.zcat where that row's ra and dec (cat.ra and cat.dec from the catalog) 
##  are within radius 1deg of the given RA and DEC we got above for M51 
##  (coord.ra.deg and coord.dec.deg from our variables defined above), and where 
##  the bmag column is less than 14.  
query = """SELECT ra, dec, Radial_Velocity, radial_velocity_error, bmag, morph_type FROM public.zcat as cat where 
    contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',{},{},1.0))=1 and
    cat.bmag < 14
    order by cat.radial_velocity_error 
    """.format(coord.ra.deg, coord.dec.deg)

In [8]:
results = heasarc_tap_services[0].service.run_async(query)
#results=heasarc_tap_services[0].search(query) 
results.to_table()

ra,dec,radial_velocity,radial_velocity_error,bmag,morph_type
float64,float64,int32,int16,float32,int16
202.4682,47.1981,474,23,9.03,4
202.4949,47.2679,558,23,10.94,0
202.5474,46.67,2569,25,13.3,-5


See the __[information on the zcat](https://heasarc.gsfc.nasa.gov/W3Browse/galaxy-catalog/zcat.html)__ for column information. (We will use the 'radial_velocity' column rather than the 'redshift' column.) We note that spiral galaxies have morph_type between 1 - 9. 

Therefore, we can generalize the query above to complete our exercise and select the brightest (bmag < 14), nearby (radial velocity < 3000), spiral ( morph_type = 1 - 9) galaxies as follows: 

In [9]:
query = """SELECT ra, dec, Radial_Velocity, radial_velocity_error, bmag, morph_type FROM public.zcat as cat where 
    cat.bmag < 14 and cat.morph_type between 1 and 9 and cat.Radial_Velocity < 3000 
    order by cat.Radial_velocity 
    """.format(coord.ra.deg, coord.dec.deg)

In [10]:
results = heasarc_tap_services[0].service.run_async(query)
#results = heasarc_tap_services[0].search(query)
results.to_table()

ra,dec,radial_velocity,radial_velocity_error,bmag,morph_type
float64,float64,int32,int16,float32,int16
10.6847,41.2688,-297,1,4.3,3
189.2076,13.1627,-223,18,10.58,2
186.7368,15.0478,-182,12,12.23,1
23.4622,30.6602,-180,1,6.5,5
186.4955,15.671,-155,26,13.7,7
183.4511,14.9001,-98,15,11.0,2
183.913,13.9013,-84,27,12.08,4
148.8993,69.063,-40,5,7.75,2
68.2119,71.8887,-28,10,12.1,7


## Synchronous versus asynchronous queries

There is one technical detail about TAP queries that you will need to know.  In the code cells above, there are two commands for sending the query, one of which is commented out.  This is because, with the TAP, there are two ways to send such queries.  The default when you use the `search()` method is to us a synchronous query, which means that the query is sent and the client waits for the response.  For large and complicated queries, this may time out, so there is another option.  The method `service.run_async()` uses an asynchronous query, which means that the query is sent, and then (under the hood without you needing to do anything), the method checks for a response.  From your point of view, these methods look the same;  PyVO is doing different things under the hood, but the method will not return until it has your result.  

You need to know about these two methods for a couple of reasons.  First, some services will limit synchronous queries, i.e. they will not necessarily return *all* the results if there are too many of them.  An asynchronous query should have no such restrictions.  In the case of the HEASARC service that we use above, it does not matter, but you should be aware of this and be in the habit of using the asynchronous queries for complete results after an initial interactive exploration.  

The second reason to be aware of this is that asynchronous queries may be queued by the service, and they can take a lot longer if the service is very busy or the job is very large.  (The synchronous option in this case may either time out, or it may return quickly but with incomplete results.)

For very large queries, you may wish to use the `submit_job()`, `wait()`, and `fetch_results()` methods to avoid locking up your Python session, as described in the [pyvo documentation](https://pyvo.readthedocs.io/en/latest/dal/index.html#jobs).


<a id="cc"></a>
# 4. TAP:  Using the TAP to cross-correlate our objects with a catalog

TAP can also be a powerful way to collect a lot of useful information from existing catalogs in one quick step. For this exercise, we will start with a list of sources, uploaded from our own table, and do a 'cross-correlation' with the *zcat* table. 

For more on creating and working with VO tables, see that [notebook](CS_VO_Tables.ipynb).  Here, we just read one in that's already prepared:  


(These take a while, i.e. half a minute.)

In [11]:
query="""
    SELECT cat.ra, cat.dec, Radial_Velocity, bmag, morph_type
    FROM zcat cat, tap_upload.mysources mt 
    WHERE
    contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',mt.ra,mt.dec,0.01))=1
    and Radial_Velocity > 0
    ORDER by cat.ra"""
zcattable = heasarc_tap_services[0].service.run_async(query, uploads={'mysources': 'data/my_sources.xml'})
#zcattable = heasarc_tap_services[0].search(query, uploads={'mysources': 'data/my_sources.xml'})
mytable = zcattable.to_table()
mytable

ra,dec,radial_velocity,bmag,morph_type
float64,float64,int32,float32,int16
136.0007,21.9679,3093,13.8,20
146.7033,22.0183,7446,14.6,-1
146.7033,22.0183,7597,15.0,-2
148.7781,14.2961,7194,15.1,2
175.0394,15.3273,3325,14.4,3
191.542,30.7323,6651,15.0,20
191.5474,30.7234,6517,14.7,-2
194.9129,28.8954,6093,15.0,0
206.5716,43.8505,2229,15.0,-1


Therefore we now have the Bmag, morphological type and radial velocities for all the sources in our list with a single TAP query. 

<a id="combo"></a>

# 5.  Combining data from different catalogs and cross-correlating

Our input list of sources contains galaxy pair candidates that may be interacting with each other. Therefore it would be interesting to know what the morphological type and the Bmagnitude are for the potential companions. 

In this advanced example, we want our search to be physically motivated since the criterion for galaxy interaction depends on the physical separation of the galaxies. Unlike the previous case, the search radius is not a constant, but varies for each candidate by the distance to the source. Specifically, we want to search for companions that are within 50 kpc of the candidate and therefore first need to find the angular diameter distance that corresponds to galaxy's distance (in our case the radial velocity).

Therefore, we begin by taking our table of objects and adding an angDdeg column:

In [12]:
## The column 'radial_velocity' is c*z but doesn't include the unit; it is km/s
## Get the speed of light from astropy.constants and express in km/s
c = const.c.to(u.km/u.s).value 
redshifts = mytable['radial_velocity']/c 
mytable['redshift'] = redshifts
physdist = 0.05*u.Mpc # 50 kpc physical distance

angDdist = Planck15.angular_diameter_distance(mytable['redshift'])
angDrad = np.arctan(physdist/angDdist)
mytable['angDdeg'] = angDrad.to(u.deg)
mytable

ra,dec,radial_velocity,bmag,morph_type,redshift,angDdeg
,,,,,,deg
float64,float64,int32,float32,int16,float64,float64
136.0007,21.9679,3093,13.8,20,0.010317137464478843,0.06354147623734327
146.7033,22.0183,7446,14.6,-1,0.024837182528454405,0.026865139047937883
146.7033,22.0183,7597,15.0,-2,0.02534086431220361,0.026347225109192074
148.7781,14.2961,7194,15.1,2,0.02399660100855506,0.027777901534139626
175.0394,15.3273,3325,14.4,3,0.011091006165338556,0.0591638655187596
191.542,30.7323,6651,15.0,20,0.022185347971629094,0.02997982632048326
191.5474,30.7234,6517,14.7,-2,0.02173837208406357,0.030579669183669427
194.9129,28.8954,6093,15.0,0,0.020324060320423405,0.032651528395800614


This time, rather than write the table to disk, we'll keep it in memory and give Tap.query() a "file-like" object using io.BytesIO():

In [13]:
## In memory only, use an IO stream. 
vot_obj=io.BytesIO()
apvot.writeto(apvot.from_table(mytable),vot_obj)
## (Reset the "file-like" object to the beginning.)
vot_obj.seek(0)

0

Now we construct and run a query that uses the new angDdeg column in every row search. Note, we also don't want to list the original candidates since we know these are in the catalog and we want rather to find any companions. Therefore, we exclude the match if the radial velocities match exactly.

This takes half a minute:

In [14]:
query="""SELECT mt.ra, mt.dec, cat.ra, cat.dec, cat.Radial_Velocity, cat.morph_type, cat.bmag 
    FROM zcat cat, tap_upload.mytable mt 
    WHERE
    contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',mt.ra,mt.dec,mt.angDdeg))=1
    and cat.Radial_Velocity > 0 and cat.radial_velocity != mt.radial_velocity
    ORDER by cat.ra"""
#  Currently broken due to a bug.
#mytable2 = heasarc_tap_services[0].service.run_async(query, uploads={'mytable':vot_obj})
mytable2 = heasarc_tap_services[0].search(query, uploads={'mytable':vot_obj})
vot_obj.close()
mytable2.to_table()

ra,dec,ra2,dec2,radial_velocity,morph_type,bmag
float64,float64,float64,float64,int32,int16,float32
146.7033,22.0183,146.70334308,22.01827217,7597,-2,15.0
146.7033,22.0183,146.70334308,22.01827217,7446,-1,14.6
175.0394,15.3273,175.05522126,15.34280243,3299,3,13.1
191.5474,30.7234,191.54198643,30.73226569,6651,20,15.0
191.542,30.7323,191.54739911,30.72338192,6517,-2,14.7
206.5716,43.8505,206.57996426,43.84385707,25864,--,18.5
206.5716,43.8505,206.59810299,43.87222974,2420,4,12.72
219.9665,42.7423,220.03853841,42.77894885,2545,7,14.0
333.8217,37.2992,333.84683658,37.28260898,5984,-3,15.06


#### Therefore, by adding new information to our original data table, we could cross-correlate with the TAP.  We find that, in our candidate list, there is one true pair of galaxies!